# Building a Chatbot

In this project, we will build a chatbot using conversations from Cornell University's [Movie Dialogue Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). The main features of our model are LSTM cells, a bidirectional dynamic RNN, and decoders with attention. 

The conversations will be cleaned rather extensively to help the model to produce better responses. As part of the cleaning process, punctuation will be removed, rare words will be replaced with "UNK" (our "unknown" token), longer sentences will not be used, and all letters will be in the lowercase. 

With a larger amount of data, it would be more practical to keep features, such as punctuation. However, I am using FloydHub's GPU services and I don't want to get carried away with too training for too long.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np
import tensorflow as tf
import time
import os

#Local libraries
import metrics
import loss_functions
import embedding_models

SEED = 1
np.random.seed(SEED)
tf.set_random_seed(SEED)


tf.__version__

'1.7.0'

## Load and Preprocess Data

In [3]:
corpus_dir = os.path.join("corpora")
var_names = ["train_prompts", "train_answers", "valid_prompts", "valid_answers", "vocab"]
file_names = [os.path.join(corpus_dir, var_name + ".txt") for var_name in var_names]

for (file_name, var_name) in zip(file_names, var_names):
    with open(file_name, "r", encoding="utf-8") as r:
        text = [ [token for token in line.strip().split(" ")] for line in r.readlines()]
        exec("{} = {}".format(var_name, text))
        


In [4]:
vocab2int = {pair[0]:int(pair[1]) for pair in vocab}
int2vocab = {index:word for (word, index) in vocab2int.items()}
(questions_vocab_to_int, questions_int_to_vocab) = (vocab2int, int2vocab)
(prompts_vocab_to_int, prompts_int_to_vocab) = (vocab2int, int2vocab) #Alternative names to ease the transition


(answers_vocab_to_int, answers_int_to_vocab) = (vocab2int, int2vocab)

UNK = vocab[0][0]
METATOKEN_INDEX = len(vocab2int)
META = "<META>"
EOS = "<EOS>"
PAD = "<PAD>"
GO = "<GO>"
codes = [META, EOS, PAD, GO]


In [5]:
def int_to_text(sequence, int2vocab):
    return [int2vocab[index] for index in sequence if index != METATOKEN_INDEX]

def text_to_int(sequence, vocab2int):
    return [vocab2int.get(token, vocab2int[UNK]) for token in sequence if token not in codes]

In [6]:
train_prompts_int = [text_to_int(prompt, questions_vocab_to_int) for prompt in train_prompts]
train_answers_int = [text_to_int(answer, answers_vocab_to_int) for answer in train_answers]
valid_prompts_int = [text_to_int(prompt, questions_vocab_to_int) for prompt in valid_prompts]
valid_answers_int = [text_to_int(answer, answers_vocab_to_int) for answer in valid_answers]

In [7]:
for i in range(10):
    print(train_prompts[i])
    print(train_prompts_int[i])
for i in range(10):
    print(train_answers[i])
    print(train_answers_int[i])

['oh', ',', 'shell', ',', 'that', 'is', 'ridiculous', ',', 'where', 'are', 'my', 'needle', '<UNK>', '<UNK>', '?']
[73, 2, 4774, 2, 13, 8, 1061, 2, 80, 14, 32, 4855, 0, 0, 5]
['sha', 'la', 'la', 'la', 'that', 'is', 'not', 'no', 'crime', '.']
[7232, 1486, 1486, 1486, 13, 8, 9, 30, 1037, 1]
['uh-huh', '.']
[724, 1]
['somebody', 'could', 'not', 'wait', 'until', 'dark', '.']
[316, 79, 9, 206, 345, 714, 1]
['that', 'is', 'it', '.']
[13, 8, 11, 1]
['i', 'could', 'almost', 'forgive', 'you', 'if', 'what', 'you', 'did', 'to', 'steve', 'came', 'from', 'jealousy', 'and', 'love', '.']
[4, 79, 505, 893, 3, 53, 16, 3, 39, 7, 1876, 246, 87, 9438, 15, 143, 1]
['certainly', 'not', '!']
[642, 9, 18]
['oh', ',', 'do', 'you', 'like', 'it', '?', 'i', 'am', 'not', 'partial', 'to', '<UNK>', ',', 'but', 'this', 'is', 'excellent', '.']
[73, 2, 12, 3, 41, 11, 5, 4, 23, 9, 5997, 7, 0, 2, 37, 26, 8, 1540, 1]
['not', 'so', '.', 'easily', '.', 'why', 'are', 'not', 'you', '.', '<UNK>', '?', 'go', 'on', '.', 'we', 'ar

## Word2Vec Embeddings

In [8]:
combined_corpus = train_prompts + train_answers + valid_prompts + valid_answers
len(combined_corpus)

417470

In [9]:
combined_corpus[:5]

[['oh',
  ',',
  'shell',
  ',',
  'that',
  'is',
  'ridiculous',
  ',',
  'where',
  'are',
  'my',
  'needle',
  '<UNK>',
  '<UNK>',
  '?'],
 ['sha', 'la', 'la', 'la', 'that', 'is', 'not', 'no', 'crime', '.'],
 ['uh-huh', '.'],
 ['somebody', 'could', 'not', 'wait', 'until', 'dark', '.'],
 ['that', 'is', 'it', '.']]

In [10]:
from gensim.models import Word2Vec
embedding_size = 1024
model = Word2Vec(sentences=combined_corpus, size=embedding_size, window=5, min_count=1, workers=4, sg=0)

In [11]:
model.wv['well'].shape

(1024,)

In [12]:
wordVecs = model.wv

In [13]:
word_vecs = np.zeros((len(model.wv.vocab),1024))
for i,word in enumerate(model.wv.index2word):
        word_vecs[vocab2int[word]] = model[word]
      

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
print("Vocabulary lengths")
print(len(word_vecs))
print(len(prompts_vocab_to_int))
print(len(answers_vocab_to_int))
print(len(prompts_int_to_vocab))
print(len(answers_int_to_vocab))
print(len(word_vecs))

Vocabulary lengths
12001
12001
12001
12001
12001
12001


In [15]:
np.save('word_Vecs.npy',word_vecs)

### Affective Embeddings

In [16]:
#Choose from embedding_models.[appended_vad, counterfitted, retrofitted]
embedding_model_path = embedding_models.appended_vad(model, vocab2int, regenerate=True)

6848/12001 words assigned corresponsing VAD values.
5153/12001 words assigned the neutral VAD vector.


## Additional Preprocessing

In [17]:
#Add EOS tokens to target data now that the embeddings have been trained
def append_eos(answers_text, answers_int):
    appended_text = [sequence + [EOS] for sequence in answers_text]
    appended_ints = [sequence + [METATOKEN_INDEX] for sequence in answers_int]
    return (appended_text, appended_ints)

(train_answers, train_answers_int) = append_eos(train_answers, train_answers_int)
(valid_answers, valid_answers_int) = append_eos(valid_answers, valid_answers_int)

print(train_answers[:5])
print(train_answers_int[:5])

[['listen', ',', 'if', 'a', 'boy', 'wants', 'to', 'do', 'homework', 'with', 'you', ',', 'it', 'really', 'means', 'he', 'did', 'not', 'have', 'nerve', 'to', 'ask', 'you', 'out', 'on', 'a', 'date', ',', 'so', 'you', 'shall', 'get', 'your', 'homework', 'together', 'and', 'pretend', 'to', 'be', 'studying', ',', 'and', 'the', 'next', 'thing', 'you', 'know', 'you', 'are', 'ordering', 'pizza', 'and', 'talking', 'about', 'your', 'favorite', 'movie', 'stars', '.', '<EOS>'], ['no', 'no', 'no', ',', 'no', 'no', '.', '<EOS>'], ['her', 'life', 'is', 'in', 'your', 'hands', '.', '<EOS>'], ['h', "'m", '?', '<EOS>'], ['why', 'did', 'not', 'they', 'say', 'that', 'before', 'you', 'started', '?', '<EOS>']]
[[225, 2, 53, 10, 254, 327, 7, 12, 2766, 36, 3, 2, 11, 108, 475, 24, 39, 9, 20, 1853, 7, 207, 3, 50, 34, 10, 662, 2, 51, 3, 55, 46, 29, 2766, 309, 15, 1678, 7, 33, 3523, 2, 15, 6, 283, 125, 3, 28, 3, 14, 6098, 2789, 15, 197, 43, 29, 1169, 587, 1650, 1, 12001], [30, 30, 30, 2, 30, 30, 1, 12001], [69, 159

## Model

In [18]:
def process_decoding_input(target_data, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat( [tf.fill([batch_size, 1], METATOKEN_INDEX), ending], 1)
    return dec_input


In [19]:
def dropout_cell(rnn_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    return tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)

def multi_dropout_cell(rnn_size, keep_prob, num_layers):    
    return tf.contrib.rnn.MultiRNNCell( [dropout_cell(rnn_size, keep_prob) for _ in range(num_layers)] )

In [20]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_lengths):
    """
    Create the encoding layer
    
    Returns a tuple `(outputs, output_states)` where
      outputs is a 2-tuple of vectors of dimensions [sequence_length, rnn_size] for the forward and backward passes
      output_states is a 2-tupe of the final hidden states of the forward and backward passes
    
    """
    forward_cell = multi_dropout_cell(rnn_size, keep_prob, num_layers)
    backward_cell = multi_dropout_cell(rnn_size, keep_prob, num_layers)
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw = forward_cell,
                                                   cell_bw = backward_cell,
                                                   sequence_length = sequence_lengths,
                                                   inputs = rnn_inputs,
                                                    dtype=tf.float32)
    return outputs, states

## Decoding

In [21]:
def decoding_layer(enc_state, enc_outputs, dec_embed_input, dec_embeddings, #Inputs
                        attn_size, rnn_size, num_layers, output_layer, #Architecture
                        keep_prob,  #Hypeparameters
                        source_lengths, target_lengths, batch_size): 
   
    with tf.variable_scope("decoding") as scope:
        dec_cell = multi_dropout_cell(rnn_size, keep_prob, num_layers)
        init_dec_state_size = batch_size
        attn_mech = tf.contrib.seq2seq.BahdanauAttention(num_units=attn_size, memory=enc_outputs,
                                                         memory_sequence_length=source_lengths)
        attn_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell, attn_mech,
                                                    attention_layer_size=dec_cell.output_size)
        init_dec_state = attn_cell.zero_state(init_dec_state_size, tf.float32).clone(cell_state=enc_state)
        
        decoder_gen = lambda helper: tf.contrib.seq2seq.BasicDecoder(attn_cell, helper, init_dec_state,
                                        output_layer = output_layer)
        
        #TRAINING
        train_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_lengths)
        train_decoder = decoder_gen(train_helper)
        train_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(train_decoder, impute_finished=True, scope=scope)
        train_logits = train_outputs.rnn_output

        #INFERENCE
        infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                                start_tokens = tf.tile([METATOKEN_INDEX],
                                                                                       [batch_size]),
                                                                 end_token = METATOKEN_INDEX)
        infer_decoder = decoder_gen(infer_helper)
        infer_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(infer_decoder, scope=scope, maximum_iterations=tf.round(tf.reduce_max(source_lengths) * 2))
        infer_ids = infer_outputs.sample_id
                
    return train_logits, infer_ids

In [22]:
def seq2seq_model(enc_embed_input, dec_embed_input, dec_embeddings, #Inputs
                  source_lengths, target_lengths, batch_size, #Dimensions
                  attn_size, rnn_size, num_layers, output_layer, #Architecture
                  keep_prob): #Hyperparameters
    
    enc_outputs, enc_states = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, source_lengths)    
    concatenated_enc_output = tf.concat(enc_outputs, -1)
    init_dec_state = enc_states[0]    
    
    
    train_logits, infer_ids = decoding_layer(init_dec_state,
                            concatenated_enc_output,
                            dec_embed_input,
                            dec_embeddings,
                            attn_size,
                            rnn_size, 
                            num_layers,
                            output_layer,
                            keep_prob,
                            source_lengths,
                            target_lengths, 
                            batch_size
                            )
    
    
    return train_logits, infer_ids

In [59]:
#FLAGS
flag_affect_function = True
flag_vad_values = True # Set to true if using VAD values appended onto existing embeddings
affect_function = "max_affective_content" #Other valid values "max_affective_dissonance", "min_affective_dissonance"

#Settings used by Asghar et al.
rnn_size = 1024
num_layers = 1
attention_size = 256
epochs_before_affective_loss = 40
epochs = 50
train_batch_size = 64
lambda_param_max_affective_content = 0.5
lambda_param_min_affective_dissonance=0.5
lambda_param_max_affective_dissonance = 0.4

#Training
learning_rate = 0.0001
keep_probability = 0.75

#Validation
valid_batch_size = 64


wordVecs = np.load(embedding_model_path).astype(np.float32)

embedding_size = wordVecs.shape[1] #Dynamically determine embedding size from loaded embedding file

metatoken_embedding = np.zeros((1, embedding_size), dtype=wordVecs.dtype)
wordVecsWithMeta = np.concatenate( (wordVecs, metatoken_embedding), axis=0 )
vocab_size_with_meta = wordVecsWithMeta.shape[0]

print("vocab_size_with_meta =", vocab_size_with_meta)
print("METATOKEN_INDEX =", METATOKEN_INDEX)
print("wordVecsWithMeta.shape =", wordVecsWithMeta.shape)
print("wordVecsWithMeta[METATOKEN_INDEX] =", wordVecsWithMeta[METATOKEN_INDEX])


vocab_size_with_meta = 12002
METATOKEN_INDEX = 12001
wordVecsWithMeta.shape = (12002, 1027)
wordVecsWithMeta[METATOKEN_INDEX] = [0. 0. 0. ... 0. 0. 0.]


In [60]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()


#                                      batch_size, sequence_length
input_data = tf.placeholder(tf.int32, [None,       None], name='input')
targets = tf.placeholder(tf.int32,    [None,       None], name='targets')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

#Determines whether we use a normal loss function or an affective loss function
train_affect = tf.placeholder(tf.bool, shape=(), name="train_affect")


#                                          batch_size
source_lengths = tf.placeholder(tf.int32, [None], name="source_lengths")
target_lengths = tf.placeholder(tf.int32, [None], name="target_lengths")
batch_size = tf.shape(input_data)[0]

full_embeddings = tf.Variable(wordVecsWithMeta,trainable=False,name="Weight")
enc_embed_input = tf.nn.embedding_lookup(full_embeddings, input_data)
dec_embed_input = tf.nn.embedding_lookup(full_embeddings, process_decoding_input(targets, batch_size))

output_layer = tf.layers.Dense(vocab_size_with_meta,bias_initializer=tf.zeros_initializer(),activation=tf.nn.relu)


# Create the training and inference logits
train_logits, infer_ids = \
seq2seq_model(enc_embed_input, dec_embed_input, full_embeddings,
        source_lengths, target_lengths, batch_size, 
        attention_size, rnn_size, num_layers, output_layer,
        keep_prob)


# Find the shape of the input data for sequence_loss
with tf.name_scope("optimization"): 
    
    mask = tf.sequence_mask(target_lengths, dtype=tf.float32)
    xent = loss_functions.cross_entropy(train_logits, targets, mask)
    perplexity = tf.contrib.seq2seq.sequence_loss(train_logits, targets, mask, metrics.perplexity)
    
    
    
    vad_values = full_embeddings[:, -3:]
    input_vad_values =enc_embed_input[:,:,1024:1027]

    if flag_affect_function:
        if affect_function == "max_affective_content":
            assert flag_vad_values
            emot_embeddings = full_embeddings[:, -3:]
            neutral_vector = tf.constant([5.0, 1.0, 5.0], dtype=tf.float32)
            affective_loss = loss_functions.max_affective_content(lambda_param_max_affective_content,
                                                                train_logits, targets,
                                                                emot_embeddings,neutral_vector, mask)
        else:
            if flag_vad_values:
                emot_embeddings = full_embeddings[:, -3]
                enc_emot_embed_input = enc_embed_input[:, -3]
            else:
                emot_embeddings = full_embeddings
                enc_emot_embed_input = enc_embed_input
            
            if affect_function == "max_affective_dissonance":
                affective_loss = loss_functions.max_affective_dissonance(lambda_param_max_affective_dissonance,
                                                                train_logits, targets, 
                                                                 enc_emot_embed_input,emot_embeddings, mask)
            else:
                affective_loss = loss_functions.min_affective_dissonance(lambda_param_min_affective_dissonance,
                                                                train_logits, targets, 
                                                                 enc_emot_embed_input,emot_embeddings, mask)
    else:
        affective_loss = xent #Just so we have a term for below 
    
    train_cost = tf.cond(train_affect, true_fn=lambda: affective_loss, false_fn=lambda: xent)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(train_cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)


### Subroutines for Sampling Output

In [61]:
def show_response(prompt_int, prediction, answer_int = None):
    pad_q = METATOKEN_INDEX
    print("Prompt")
    print("  Word Ids: {}".format([i for i in prompt_int if i != pad_q]))
    print("      Text: {}".format(int_to_text(prompt_int, prompts_int_to_vocab)))
    
    pad_a = METATOKEN_INDEX
    if answer_int is not None:
        print("Target Answer")
        print("  Word Ids: {}".format([i for i in answer_int if i != pad_a]))
        print("      Text: {}".format(int_to_text(answer_int, answers_int_to_vocab)))

    print("\nPrediction")
    print('  Word Ids: {}'.format([i for i in prediction if i != pad_a]))
    print('      Text: {}'.format(int_to_text(prediction, answers_int_to_vocab)))
        
def check_response(session, prompt_int, answer_int=None):
    """
    session - the TensorFlow session
    question_int - a list of integers
    answer - the actual, correct response (if available)
    """
    
    two_d_prompt_int = [prompt_int]
    p_lengths = [len(prompt_int)]
    
    [infer_ids_output] = session.run([infer_ids], feed_dict = {input_data: np.array(two_d_prompt_int, dtype=np.float32),
                                                      source_lengths: p_lengths,
                                                      keep_prob: 1})
    
    show_response(prompt_int, infer_ids_output[0], answer_int)

In [62]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    pad_int = METATOKEN_INDEX
    max_sentence_length = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence_length - len(sentence)) for sentence in sentence_batch]

In [63]:
def batch_data(questions, answers, batch_size):
    """Batch questions and answers together"""
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        
        source_lengths = np.array( [len(sentence) for sentence in questions_batch] )
        target_lengths = np.array( [len(sentence) for sentence in answers_batch])
        
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        yield source_lengths, target_lengths, pad_questions_batch, pad_answers_batch

In [64]:
def parallel_shuffle(source_sequences, target_sequences):
    if len(source_sequences) != len(target_sequences):
        raise ValueError("Cannot shuffle parallel sets with different numbers of sequences")
    indices = np.random.permutation(len(source_sequences))
    shuffled_source = [source_sequences[indices[i]] for i in range(len(indices))]
    shuffled_target = [target_sequences[indices[i]] for i in range(len(indices))]
    
    return (shuffled_source, shuffled_target)

### Training Loop Options

In [65]:
#TRAINING
display_step = 100 # Check training loss after every 100 batches

#VALIDATION
validation_check = ((len(train_prompts))//train_batch_size//2)-1 #Check validation loss every half-epoch
#Minimum number of epochs before we start checking sample output
min_epochs_before_validation = 2

#Used to make uniquely directories, not to identify when a model is saved
time_string = time.strftime("%b%d_%H:%M:%S")

checkpoint_dir = os.path.join("checkpoints", time_string)
if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)
checkpoint_best = str(checkpoint_dir) + "/" + "best_model.ckpt" 
checkpoint_latest = str(checkpoint_dir) + "/" + "latest_model.ckpt"

log_dir = os.path.join("logging", time_string)
if not os.path.exists(log_dir): os.makedirs(log_dir)
train_log = os.path.join(log_dir, "train.csv")
valid_log = os.path.join(log_dir, "valid.csv")



### Logging Progress

In [66]:
def log_entries(csv_path, *fields, header = False):
    if len(fields[0]) < 1: return
    mode = "w" if header else "a"
    with open(csv_path, mode, encoding="utf-8") as log:
        lines = []
        num_lines = len(fields[0])
        lines = "\n".join(",".join([str(field[i]) for field in fields]) 
                          for i in range(num_lines)
        )
        log.write(lines + "\n")

In [67]:
def clear_fields(log_fields):
    for field in log_fields:
        field.clear()

In [ ]:
train_epoch_nos = []
train_batch_tokens = [] #Number of tokens in a batch
train_batch_losses = [] #Per-token loss for a batch
train_log_fields = [train_epoch_nos, train_batch_tokens, train_batch_losses]

valid_epoch_nos = []
valid_check_nos = []
valid_batch_tokens = []
valid_batch_losses = []
valid_log_fields = [valid_epoch_nos, valid_check_nos, valid_batch_tokens, valid_batch_losses]

train_start_time = None #For logging time for sets of batches
log_entries(train_log, ["epoch"], ["num_tokens"], ["loss_per_token"], header=True)
log_entries(valid_log, ["epoch"], ["check"], ["num_tokens"], ["perplexity_per_token"], header=True)
print("Initialized empty training log {}, validation log {}".format(train_log, valid_log))

best_valid_loss = float("inf")
use_affect_func = False

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.train.Saver().save(sess, checkpoint_latest)
    print("Initialized model parameters, wrote initial model to {}".format(checkpoint_latest))
    if not use_affect_func: print("Beginning training with cross-entropy loss.")
    else:                   print("Beginning training with {}".format(affect_function))
    for epoch_i in range(1, epochs+1):
        if not use_affect_func and epoch_i > epochs_before_affective_loss:
            print("Switching from cross-entropy loss to {}".format(affect_function))
            use_affect_func = True        
        
        print("Shuffling training data . . .")
        (train_prompts_int, train_answers_int) = parallel_shuffle(train_prompts_int, train_answers_int)
        
        valid_check_no = 1
        
        
        for batch_i, (p_lengths, a_lengths, prompts_batch, answers_batch) in enumerate(
                batch_data(train_prompts_int, train_answers_int, train_batch_size)):
            if train_start_time is None: train_start_time = time.time()
            
            #VALIDATION CHECK
            if batch_i % validation_check == 0 and epoch_i > min_epochs_before_validation:
                print("Shuffling validation data . . .")
                (valid_prompts_int, valid_answers_int) = parallel_shuffle(valid_prompts_int, valid_answers_int)
                
                clear_fields(valid_log_fields)

                
                valid_start_time = time.time()
                for batch_ii, (p_lengths, a_lengths, prompts_batch, answers_batch) in \
                        enumerate(batch_data(valid_prompts_int, valid_answers_int, valid_batch_size)):

                    [valid_loss] = sess.run([perplexity],
                        {input_data: prompts_batch, targets: answers_batch,
                        source_lengths: p_lengths, target_lengths: a_lengths, keep_prob: 1})
                    valid_epoch_nos.append(epoch_i)
                    valid_check_nos.append(valid_check_no)
                    valid_batch_tokens.append(sum(a_lengths))
                    valid_batch_losses.append(valid_loss)

                
                valid_check_no += 1
                duration = time.time() - valid_start_time
                avg_valid_loss = sum(loss*tokens 
                        for (loss, tokens) in zip(valid_batch_losses, valid_batch_tokens)) / sum(valid_batch_tokens)
                
                log_entries(valid_log, *(valid_log_fields))
                clear_fields(valid_log_fields)
                print("Processed validation set in {:>4.2f} seconds".format(duration))
                print("Average perplexity per token = {}".format(avg_valid_loss))
                if avg_valid_loss >= best_valid_loss:
                    print("No improvement for validation loss.")
                else:
                    best_valid_loss = avg_valid_loss
                    print("New record for validation loss!")
                    print("Saving best model to {}".format(checkpoint_best))
                    tf.train.Saver().save(sess, checkpoint_best)
                check_response(sess, prompts_batch[-1], answers_batch[-1])
                
                train_start_time = time.time()
            
            #TRAINING
            _, loss = sess.run([train_op, train_cost],
                {input_data: prompts_batch, targets: answers_batch,
                 source_lengths: p_lengths, target_lengths: a_lengths,
                 keep_prob: keep_probability,
                 train_affect: use_affect_func})
            train_epoch_nos.append(epoch_i)
            train_batch_losses.append(loss)
            train_batch_tokens.append(sum(a_lengths))
            
            if batch_i % display_step == 0:
                duration = time.time() - train_start_time
                avg_train_loss = sum(loss*tokens 
                        for (loss, tokens) in zip(train_batch_losses, train_batch_tokens)) / sum(train_batch_tokens)
                    
                log_entries(train_log, *(train_log_fields))
                clear_fields(train_log_fields)
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss-per-Token: {:>9.6f}, Seconds: {:>4.2f}'
                      .format(epoch_i, epochs, batch_i, len(train_prompts_int) // train_batch_size, 
                              avg_train_loss, duration),
                         flush=True)
                train_start_time = time.time()

        print("{} epochs completed, saving model to {}".format(epoch_i, checkpoint_latest))
        tf.train.Saver().save(sess, checkpoint_latest)
        log_entries(train_log, *(train_log_fields))
        clear_fields(train_log_fields)



Initialized empty training log logging/Jun26_13:26:03/train.csv, validation log logging/Jun26_13:26:03/valid.csv
Initialized model parameters, wrote initial model to checkpoints/Jun26_13:26:03/latest_model.ckpt
Beginning training with cross-entropy loss.
Shuffling training data . . .
Epoch   1/50 Batch    0/2556 - Loss-per-Token:  9.396128, Seconds: 1.44


In [ ]:
def question_to_seq(question, vocab_to_int, int_to_vocab):
    '''Prepare the question for the model'''
    cleaned_question = Corpus.clean_sequence(question)
    return [vocab_to_int.get(word, vocab_to_int[UNK]) for word in cleaned_question]


In [ ]:
# Use a question from the data as your input
random = np.random.choice(len(train_prompts_int))
prompt_int = train_prompts_int[random]
answer_int = train_answers_int[random]

saver = tf.train.Saver()
with tf.Session() as sess:
    # Run the model with the input question
    saver.restore(sess, checkpoint)
    check_response(sess, prompt_int, answer_int, best_only=False)
    
